In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import bqplot
import ipywidgets

data_url = 'https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/licenses_fall2022.csv'
df_licenses = pd.read_csv(data_url)
df_licenses.head()

,_id,License Type,Description,License Number,License Status,Business,Title,First Name,Middle,Last Name,...,Specialty/Qualifier,Controlled Substance Schedule,Delegated Controlled Substance Schedule,Ever Disciplined,LastModifiedDate,Case Number,Action,Discipline Start Date,Discipline End Date,Discipline Reason
0,1189509,DETECTIVE BOARD,PERMANENT EMPLOYEE REGISTRATION,129446286,NOT RENEWED,N,NaN,EILEEN,NaN,SANTACRUZ,...,None,None,None,N,03/18/2022,None,None,None,None,None
1,801037,DETECTIVE BOARD,FIREARM CONTROL CARD,229030294.0,NOT RENEWED,N,NaN,DAGMAR,J,NORDLUND,...,None,None,None,N,08/16/2006,None,None,None,None,None
2,365129,COSMO,LICENSED COSMETOLOGIST,11053076.0,NOT RENEWED,N,NaN,RADOJE,NaN,ZELENOVIC,...,None,None,None,N,05/26/2006,None,None,None,None,None
3,595427,COSMO,LICENSED COSMETOLOGIST,11295645.0,ACTIVE,N,NaN,BECKY SUE,L,BURROUGHS,...,None,None,None,N,11/12/2021,None,None,None,None,None
4,653668,COSMO,LICENSED NAIL TECHNICIAN,169006247,NOT RENEWED,N,NaN,BILL G,L,LETNER,...,None,None,None,N,05/30/2006,None,None,None,None,None


In [2]:
df_licenses.columns

Index(['_id', 'License Type', 'Description', 'License Number',
       'License Status', 'Business', 'Title', 'First Name', 'Middle',
       'Last Name', 'Prefix', 'Suffix', 'Business Name', 'BusinessDBA',
       'Original Issue Date', 'Effective Date', 'Expiration Date', 'City',
       'State', 'Zip', 'County', 'Specialty/Qualifier',
       'Controlled Substance Schedule',
       'Delegated Controlled Substance Schedule', 'Ever Disciplined',
       'LastModifiedDate', 'Case Number', 'Action', 'Discipline Start Date',
       'Discipline End Date', 'Discipline Reason'],
      dtype='object')

In [3]:
df_licenses['Diff_Date'] = pd.to_datetime(df_licenses['Expiration Date'], errors='coerce',format='%m/%d/%Y') - pd.to_datetime(df_licenses['Effective Date'],errors='coerce',format='%m/%d/%Y')
df_licenses['Diff_Date'] = df_licenses['Diff_Date'].dt.days
df_licenses['Diff_Date'].fillna(0, inplace=True)

In [4]:
licenses_group = df_licenses.groupby(['License Status', 'License Type'])['Diff_Date'].mean()
uniq_status = df_licenses['License Status'].unique()
uniq_type = df_licenses['License Type'].unique()

In [5]:
licenses_group

License Status                        License Type     
ACTIVE                                ARCHITECT             753.750000
                                      ATHLETIC TRAINER      716.000000
                                      AUCTIONEER            816.000000
                                      CLIN PSYCHOLOGIST     692.777778
                                      COLLECTION AGENCY    1038.285714
                                                              ...     
Non Sufficient Fund Check Terminated  COSMO                5220.000000
                                      DETECTIVE BOARD       220.000000
TERMINATED CARD RETURNED              DETECTIVE BOARD       764.844548
TERMINATED VALID REASON               DETECTIVE BOARD       834.421053
TERMINATED WITHOUT CARD               DETECTIVE BOARD      1019.468750
Name: Diff_Date, Length: 105, dtype: float64

In [6]:
print(len(uniq_status))
print(len(uniq_type))

13
28


In [7]:
data = []
for s in uniq_status:
    status_row = np.array([licenses_group[s][t] if t in licenses_group[s].index else 0 for t in uniq_type])
    data.append(status_row)
data = np.array(data)

In [8]:
data

array([[ 1.14460619e+03,  1.61345916e+03,  5.92695652e+02,
         2.31784161e+03,  8.98840909e+02,  3.07333333e+02,
         4.11500000e+02,  5.50607143e+02,  0.00000000e+00,
         7.30294118e+02,  7.52166667e+02,  5.24133333e+02,
         1.09116667e+03,  7.39000000e+02,  5.23125000e+02,
         6.94250000e+02,  1.00780000e+03,  6.93375000e+02,
         2.73900000e+03,  2.21142857e+02,  1.35078571e+03,
         7.72500000e+02,  6.59000000e+02,  7.29500000e+02,
         1.91200000e+03,  3.97500000e+02,  8.18000000e+02,
         1.41900000e+03],
       [ 4.36724717e+03,  6.87604255e+02,  7.56357143e+02,
         1.06040506e+03,  7.49304348e+02,  1.12306667e+03,
         7.16000000e+02,  6.90189189e+02,  0.00000000e+00,
         7.48608696e+02,  7.53750000e+02,  0.00000000e+00,
         1.08528571e+03,  7.63800000e+02,  1.12960000e+03,
         1.03828571e+03,  7.05000000e+02,  6.61142857e+02,
         7.52600000e+02,  6.92777778e+02,  6.52500000e+02,
         7.73666667e+02,  7.33

In [9]:
myLabel = ipywidgets.Label()

In [10]:
def on_selected(change):
    if len(change['owner'].selected)==1: # only do things if I select a single cell
        #print(change['owner'].selected[0])
        i, j =change['owner'].selected[0]
        v = data[i,j]
        selected_type = uniq_type[j]
        selected_status = uniq_status[i]
        myLabel.value = f'{(i, j)} Selected Status: {str(selected_status)}; Selected Type: {str(selected_type)}; Data value={str(v)}'

# 2. Scales
col_sc = bqplot.ColorScale(scheme='Greens')
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# 3. Axis -- skip this for right now
col_ax = bqplot.ColorAxis(scale=col_sc, 
                         orientation='vertical',
                         side='right')
x_ax = bqplot.Axis(scale=x_sc)
y_ax = bqplot.Axis(scale=y_sc, orientation='vertical')

# 4. Mark -- grid heatmap
heat_map = bqplot.GridHeatMap(color=data, 
                              scales={'color':col_sc, 'row':y_sc, 'column':x_sc},
                             interactions={'click':'select'},
                             selected_style={'fill':'magenta'})

# 5. interactions -- skip for now as well
heat_map.observe(on_selected,'selected')

# put it all together as a figure

fig = bqplot.Figure(marks=[heat_map], axes=[col_ax, x_ax,y_ax])

#fig
myDashboard = ipywidgets.VBox([myLabel, fig])
myDashboard

In [11]:
# licenses_group['CHANGE OF OWNERSHIP']['MEDICAL BOARD']

KeyError: 'MEDICAL BOARD'